In [ ]:
import polars as pl
import os
os.chdir("/zata/zippy/ramirezc/splice-model-benchmark/golden_standard")

# GTF_PARQUET = pl.read_parquet('/zata/zippy/ramirezc/static_files/gencode.v29.basic.annotation.gtf.parquet')

# exon_gtf = GTF_PARQUET.filter(
#     pl.col('feature') == 'exon'
# ).select(
#     'seqname', 'start', 'end', 'strand'
# ).rename(
#     {'seqname': 'chrom'}
# ).sort(
#     'chrom', 'start'
# )
# forward_exon_gtf = exon_gtf.filter(pl.col('strand') == '+').drop('strand')
# reverse_exon_gtf = exon_gtf.filter(pl.col('strand') == '-').drop('strand')
# forward_exon_gtf.write_csv('forward_exons.bed', separator='\t', include_header=False,)
# reverse_exon_gtf.write_csv('reverse_exons.bed', separator='\t', include_header=False,)

In [7]:
import subprocess
import gtfparse
from pybedtools import BedTool
import os
os.getcwd
os.chdir("/zata/zippy/ramirezc/splice-model-benchmark/golden_standard")
os.environ['MPLBACKEND'] = 'Agg'

In [2]:
exons = BedTool('forward_genes.bed')
merged_exons = exons.merge()
sorted_exons = merged_exons.sort(g='genome.sizes')
blacklist = sorted_exons.complement(g='genome.sizes')
blacklist.saveas('blacklist.bed')

cmd = [
    "bamCoverage",
    "--bam", 'snakemake_output/ENCBS020DQC/ENCFF835KGV.filtered.bam',
    "--outFileName", 'coverage.bg',
    "--outFileFormat", "bedgraph",
    "--filterRNAstrand", "forward",
    "--binSize", "1",
    "--blackListFileName", 'blacklist.bed',
    "--normalizeUsing", "RPKM",
    "--numberOfProcessors", '32'
]
bamCoverage = subprocess.run(cmd, check=True)

Traceback (most recent call last):
  File "/home/ramirezc/.local/bin/bamCoverage", line 5, in <module>
    from deeptools.bamCoverage import main
  File "/home/ramirezc/.local/share/pipx/venvs/deeptools/lib/python3.10/site-packages/deeptools/bamCoverage.py", line 8, in <module>
    from deeptools import writeBedGraph  # This should be made directly into a bigWig
  File "/home/ramirezc/.local/share/pipx/venvs/deeptools/lib/python3.10/site-packages/deeptools/writeBedGraph.py", line 9, in <module>
    from deeptools.utilities import getCommonChrNames
  File "/home/ramirezc/.local/share/pipx/venvs/deeptools/lib/python3.10/site-packages/deeptools/utilities.py", line 5, in <module>
    import matplotlib as mpl
  File "/home/ramirezc/.local/share/pipx/venvs/deeptools/lib/python3.10/site-packages/matplotlib/__init__.py", line 1296, in <module>
    rcParams['backend'] = os.environ.get('MPLBACKEND')
  File "/home/ramirezc/.local/share/pipx/venvs/deeptools/lib/python3.10/site-packages/matplotlib/

CalledProcessError: Command '['bamCoverage', '--bam', 'snakemake_output/ENCBS020DQC/ENCFF835KGV.filtered.bam', '--outFileName', 'coverage.bg', '--outFileFormat', 'bedgraph', '--filterRNAstrand', 'forward', '--binSize', '1', '--blackListFileName', 'blacklist.bed', '--normalizeUsing', 'RPKM', '--numberOfProcessors', '32']' returned non-zero exit status 1.

In [6]:
!wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_48/gencode.v48.basic.annotation.gtf.gz
!gunzip gencode.v48.basic.annotation.gtf.gz
gtf = gtfparse.read_gtf('gencode.v48.basic.annotation.gtf')
gtf.write_parquet('/zata/zippy/ramirezc/static_files/gencode.v48.basic.annotation.gtf.parquet')

--2025-05-16 01:49:44--  https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_48/gencode.v48.basic.annotation.gtf.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33712235 (32M) [application/x-gzip]
Saving to: ‘gencode.v48.basic.annotation.gtf.gz’

gencode.v48.basic.a 100%[===================>]  32.15M  20.9MB/s    in 1.5s    

2025-05-16 01:49:46 (20.9 MB/s) - ‘gencode.v48.basic.annotation.gtf.gz’ saved [33712235/33712235]



INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'tag', 'transcript_id', 'transcript_type', 'transcript_name', 'exon_number', 'exon_id', 'hgnc_id', 'havana_gene', 'transcript_support_level', 'ont', 'havana_transcript', 'protein_id', 'ccdsid', 'artif_dupl']


In [10]:
import polars as pl
import bioframe as bf

intron_df = pl.read_csv('/zata/zippy/ramirezc/splice-model-benchmark/golden_standard/snakemake_output/ENCBS105DDE/ENCFF279ABL.introns.tab', separator='\t')
slim_gtf = pl.read_parquet('/zata/zippy/ramirezc/static_files/gencode.v48.basic.annotation.gtf.parquet').select(['seqname', 'start', 'end', 'strand', 'feature', 'gene_id', 'gene_name', 'gene_type'])
filtered_gtf = slim_gtf.filter(pl.col('feature') == 'gene', pl.col('gene_type') == 'protein_coding').rename({'seqname': 'chrom'})
overlaps = bf.overlap(
    filtered_gtf.to_pandas(),
    intron_df.to_pandas(),
    on=['strand'],
    how='inner',
    suffixes=('_gene', '_intron')
)

contained = overlaps[
    (overlaps['start_intron'] >= overlaps['start_gene']) & 
    (overlaps['end_intron'] <= overlaps['end_gene'])
]
contained_pl = pl.from_pandas(contained)
unique_introns = contained_pl.unique(subset=['chrom_intron', 'start_intron', 'end_intron', 'strand_intron', 'reads_intron', 'reads_per_million_intron']).rename({'chrom_intron': 'chrom', 'start_intron': 'start', 'end_intron': 'end', 'strand_intron': 'strand', 'reads_intron': 'reads', 'reads_per_million_intron': 'reads_per_million', 'gene_name_gene': 'gene_name',}).select('chrom', 'start', 'end', 'strand', 'gene_name', 'reads', 'reads_per_million').sort('chrom', 'start')
unique_introns.write_csv('test.bed', separator='\t')